# Fetch Circle Elevations
### Purpose
In this notebook I query the USGS to get elevation data for all of the circles.
This notebook addresses some one of the tasks in Github issue #35

### Author: 
Ian Davis
### Date: 
2020-03-31
### Update Date: 
2020-05-03

### Inputs 
1.1-circles_to_many_stations_usa_weather_data_20200424213015.csv - Comma separate file of the Christmas Bird Count and matches to 1 or more NOAA weather stations.
- Data Dictonary can be found here: http://www.audubon.org/sites/default/files/documents/cbc_report_field_definitions_2013.pdf
1.2-ijd-fetch-circle-elevations-OFFLINE.csv - Previously generated elevation data. This file will be used when you want to get the elevation data from an offline source and aoivd 100,000+ queries.

### Output Files
1.2-ijd-fetch-circle-elevations_20200502155633.csv - Only 1 column is added to the dataset, 'circle_elev'. This column is the elevation in meters for a given latitude and longitude of the circle centroid.

## Steps or Proceedures in the notebook 
- Set runtime options
    - Set option to retrieve elevations from offline source, or through the USGS queries
    - Set option to only test the USGS query (NOTE: running the query function for the whole dataset will take 24+ hours)
- Create a function to make a remote request to the USGS API
- Create a function to supply inputs to the remote request and return the elevation value
- Main sequence
    - Read in dataset
    - Create circl_elev column
    - Loop through the dataset in chunks of 10000 to get elevation data
    - (Optional) Retrieve elevations from offline data source instead of queries
    - Write new dataset .txt file

## References
- elevation query: https://stackoverflow.com/questions/58350063/obtain-elevation-from-latitude-longitude-coordinates-with-a-simple-python-script
- lamda functions: https://thispointer.com/python-how-to-use-if-else-elif-in-lambda-functions/
- apply on Nulls: https://stackoverflow.com/questions/26614465/python-pandas-apply-function-if-a-column-value-is-not-null

In [ ]:
# Imports 
import pandas as pd
import numpy as np
import requests
import urllib
import urllib3
import time
import gzip
import logging
import sys
from datetime import datetime

In [ ]:
# Check to see if you are running 32-bit Python (output would be False)
# 32-bit Python could result in Memory Error when reading in large dataset
import sys
sys.maxsize > 2**32

## Set File Paths and Runtime Options

In [ ]:
# Used to classify the name 
time_now = datetime.today().strftime('%Y%m%d%H%M%S')

# File paths and script options
PATH_TO_PAIRED_DATA = "../data/Cloud_Data/1.1-circles_to_many_stations_usa_weather_data_20200424213015.csv"
PATH_TO_OFFLINE_ELEVATION_DATA = "../data/Cloud_Data/1.2-ijd-fetch-circle-elevations-OFFLINE.csv"
PATH_TO_LOG_FILE = "../data/Cloud_Data/1.2-ijd-fetch_circle_elevations_"+time_now+".log"

# option to pull offline elevation data from the /attic instead of running the queries
get_offline_data = False

# option to run a simple test of the query; only 1000 rows are queried instead of full dataset
test_query = False

## Create a Log File

In [ ]:
if not get_offline_data:
    logging.basicConfig(filename=PATH_TO_LOG_FILE, 
                        filemode='w', 
                        format='%(message)s', 
                        level=logging.INFO)
    logging.info('This log file shows the row index, lat, lon\n')

## Create a function to make a remote request to the USGS API

In [ ]:
def make_remote_request(url: str, params: dict):
    """
    Makes the remote request
    Continues making attempts until it succeeds
    """

    count = 1
    while True:
        try:
            response = requests.get((url + urllib.parse.urlencode(params)))
            time.sleep(1)
        except (OSError, urllib3.exceptions.ProtocolError) as error:
            logging.info('\n')
            logging.info('*' * 20, 'Error Occured', '*' * 20)
            logging.info(f'Number of tries: {count}')
            logging.info(f'URL: {url}')
            logging.info(error)
            logging.info('\n')
            count += 1
            time.sleep(0.5)
            continue
        break

    return response

## Create a function to supply inputs to the remote request and return the elevation value

In [ ]:
def elevation_function(x):
    """
    x - longitude
    y - latitude
    returns elevation in meters
    """
    
    url = 'https://nationalmap.gov/epqs/pqs.php?'
    params = {'x': x[1],
              'y': x[0],
              'units': 'Meters',
              'output': 'json'}
    logging.info(str(x.name)+'\t\t'+str(x[0])+'\t\t'+str(x[1]))   # print row index, lat, lon
    result = make_remote_request(url, params)
    
    return result.json()['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation']

## Loops to Query the USGS API

In [ ]:
# Boolean to skip loop of queries and just pull elevation data from the "attic"
if not get_offline_data:
    
    # load paired data file
    #data_iterator = pd.read_csv(PATH_TO_TEMP_DATA, chunksize=1000, encoding = "ISO-8859-1", sep="\t")
    data_iterator = pd.read_csv(PATH_TO_PAIRED_DATA, chunksize=1000, compression='gzip')
        
    chunk_list = []  
    
    # Each chunk is in dataframe format
    for data_chunk in data_iterator:
        # create elevation column
        data_chunk.loc[:, 'circle_elev'] = np.nan
        
        # initial list of indices which are missing elevations
        missing = data_chunk.loc[data_chunk['circle_elev'].isnull()].index
    
        # while loop to go over the dataset chunk times in the event that query requests fail
        cnt=0 # counter to break while loop after
        while len(missing) > 0:
            if cnt == 5: break # exit while loop
            cnt+=1
            logging.info('while counter: '+str(cnt))
        
            try:
                # combination of apply() function and lambda() function, only on nulls (see reference links above)
                data_chunk.loc[:, 'circle_elev'] = data_chunk.loc[:, ['lat', 'lon', 'circle_elev']].apply(lambda x: elevation_function(x[0:2]) if(pd.isnull(x[2])) else x[2], axis=1)
            except:
                # on occasion query completely fails and crashes the function call
                # problem is the stack prints to the notebook
                # https://gist.github.com/wassname/d17325f36c36fa663dd7de3c09a55e74
                #logging.error("Exception occurred", exc_info=True)
                logging.info("Exception occurred")
                continue
    
            # get new list of missing indices
            missing = data_chunk.loc[data_chunk['circle_elev'].isnull()].index
            # break the loop if there are no missing elevations
            if len(missing) == 0: break
        
        # Append data chunk with elevation data to combined list
        chunk_list.append(data_chunk)
        
        # Convert combined list into dataframe
        filtered_data = pd.concat(chunk_list)
        # Intermediate writes to .csv file so work is not lost in the event of code failure
        filtered_data.to_csv("../data/Cloud_Data/1.2-ijd-fetch-circle-elevations_INT.csv", sep=',', index=False)
        del(filtered_data)
        
        # If just running a test, break the loop
        if test_query: break
    
    # Convert combined list into dataframe
    filtered_data = pd.concat(chunk_list)
    
    # close log file
    #log.close()

## (Optional) Get Elevation Data from Offline Source

In [ ]:
if get_offline_data:
    # load offline data file
    offline_data = pd.read_csv(PATH_TO_OFFLINE_ELEVATION_DATA)
    
    # convert count_date to string or merge won't match them properly 
    offline_data['count_date'] = pd.to_datetime(offline_data['count_date'])
    
    # round the latitude and longitudes to 4 digits
    offline_data['lat'] = offline_data['lat'].round(3)
    offline_data['lon'] = offline_data['lon'].round(3)
    
    # load paired data file
    #data_iterator = pd.read_csv(PATH_TO_TEMP_DATA, chunksize=10000, encoding = "ISO-8859-1", sep="\t")
    data_iterator = pd.read_csv(PATH_TO_PAIRED_DATA, 
                                compression='gzip', 
                                chunksize=10000)
        
    chunk_list = []  

    # Each chunk is in dataframe format
    print('The chunks should be the same length after merge.')
    for data_chunk in data_iterator:
        data_chunk['count_date'] = pd.to_datetime(data_chunk['count_date'])
        data_chunk['lat'] = data_chunk['lat'].round(3)
        data_chunk['lon'] = data_chunk['lon'].round(3)
        filtered_chunk = pd.merge(data_chunk, 
                                  offline_data[['lat', 'lon', 'count_date', 'count_year', 'id', 'circle_elev']], 
                                  on=['lat', 'lon', 'count_date', 'count_year', 'id'],
                                  how='left',
                                  copy=False)
        chunk_list.append(filtered_chunk)
        print('Chunk Length Before: ', data_chunk.shape)
        print('Chunk Length After: ', filtered_chunk.shape)

    
    filtered_data = pd.concat(chunk_list)

In [ ]:
# find duplicates in offline data
if get_offline_data:
    offline_data.duplicated(subset=['lat', 'lon', 'count_date', 'count_year', 'id']).value_counts()

In [ ]:
# are there any duplicates in the data chunk?
if get_offline_data:
    data_chunk.duplicated(subset=['lat', 'lon', 'count_date', 'count_year', 'id']).value_counts()

## Screen Elevation Data

In [ ]:
# Ensure elevations are the float
filtered_data = pd.concat(chunk_list)
filtered_data.loc[:, 'circle_elev'] = filtered_data.loc[:, 'circle_elev'].astype(float)

In [ ]:
# Remove bad elevation values
filtered_data.loc[filtered_data['circle_elev'] < -10000.0, 'circle_elev'] = np.nan 

In [ ]:
filtered_data[['lat', 'lon', 'count_date', 'circle_elev']].head(50)

In [ ]:
# Create histogram of elevations
filtered_data.hist(column='circle_elev')

In [ ]:
# Same number of rows? Should be 109390
len(filtered_data.index)

In [ ]:
# drop unnamed columns
# they were likely index columns auto-generated by pandas and then written to csv files, unintentionally
filtered_data = filtered_data.loc[:, ~filtered_data.columns.str.contains('^Unnamed')]

In [ ]:
# sort dataframe on existing index
filtered_data.sort_values(['int64_field_0'], ascending=[True], inplace=True)

In [ ]:
filtered_data.head()

In [ ]:
if get_offline_data:
    print('If from an offline source, check to make sure circle elevations are not being lost during merge:\n')
    print('NA in Merged:\n', filtered_data['circle_elev'].isna().value_counts())
    print('\n')
    print('NA in Offline:\n', offline_data['circle_elev'].isna().value_counts())

In [ ]:
print('Missing elevations:')
filtered_data['circle_elev'].isna().value_counts()

In [ ]:
print('How many elevations at sea level?')
filtered_data.loc[filtered_data['circle_elev'] == 0.0].shape

## Save the output

In [ ]:
filtered_data.to_csv("../data/Cloud_Data/1.2-ijd-fetch-circle-elevations_"+time_now+".csv", 
                     sep=',', 
                     compression='gzip',
                     index=False)